In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_matching_tools.io.graph_dataset import GraphDataset
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
from graph_matching_tools.algorithms.multiway.hippi import hippi_multiway_matching
from graph_matching_tools.algorithms.kernels.gaussian import create_gaussian_node_kernel
from graph_matching_tools.algorithms.kernels.utils import create_full_node_affinity_matrix
from graph_matching_tools.algorithms.multiway.stiefel import sparse_stiefel_manifold_sync
from graph_matching_tools.algorithms.multiway import quickmatch
from graph_matching_tools.algorithms.multiway import matcheig
import tools.graph_processing as gp
import scipy.io as sio

In [2]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [3]:
def remove_dummy_rowcol_X(bulk_X, graphs):
    
    dummy_mask = [list(nx.get_node_attributes(graph,'is_dummy').values()) for graph in graphs]
    dummy_mask = sum(dummy_mask,[])
    dummy_indexes = [i for i in range(len(dummy_mask)) if dummy_mask[i]==True]     
    
    bulk_X = np.delete(bulk_X,dummy_indexes,0) # delete the dummy rows
    bulk_X = np.delete(bulk_X,dummy_indexes,1) # delete the dummy columns
    
    return bulk_X

In [4]:
path_to_graph_folder = '../data/simu_graph/NEW_SIMUS_JULY_11/'
path_to_dummy_graphs_folder = '../data/simu_graph/NEW_SIMUS_JULY_11_with_dummy/'

In [5]:
trials = np.sort(os.listdir(path_to_graph_folder))

sigma = 200

udim = 101

scores = {100:[],200:[],400:[],1000:[]}
precision = {100:[],200:[],400:[],1000:[]}
recall = {100:[],200:[],400:[],1000:[]}

for trial in trials:
    
    if float(trial) >= 0.0:
        
        print('trial: ', trial)

        all_files = os.listdir(path_to_graph_folder+trial)

        for folder in all_files:

            if os.path.isdir(path_to_graph_folder+trial+'/'+ folder):

                print('Noise folder: ',folder)
                
                path_to_graphs = path_to_graph_folder + '/' + trial + '/' + folder+'/graphs/'
                path_to_dummy_graphs = path_to_dummy_graphs_folder + '/' + trial + '/' + folder+'/0/graphs/'
                path_to_groundtruth_ref = path_to_graph_folder + '/' + trial +'/' + folder + '/permutation_to_ref_graph.gpickle'
                path_to_groundtruth  = path_to_graph_folder + '/' + trial + '/' + folder + '/ground_truth.gpickle'
                
            
                noise = folder.split(',')[0].split('_')[1]
#                 graph_meta = GraphDataset(path_to_graphs, path_to_groundtruth_ref)
                ground_truth =  nx.read_gpickle(path_to_groundtruth)
    
                list_graphs_dummy = gp.load_graphs_in_list(path_to_dummy_graphs)
                sizes_dummy = [nx.number_of_nodes(g) for g in list_graphs_dummy]
        
                list_graphs = gp.load_graphs_in_list(path_to_graphs)
                sizes = [nx.number_of_nodes(g) for g in list_graphs]

                res = get_permutation_matrix_from_dictionary(ground_truth, sizes)
                
#                 node_kernel = create_gaussian_node_kernel(sigma,'coord')
#                 knode = create_full_node_affinity_matrix(list_graphs, node_kernel)
                
                # Initialize using pairwise results
                path_to_X_mat = path_to_graph_folder + '/' + trial + '/' + folder
                path_X_kergm = path_to_X_mat + '/X_pairwise_kergm.mat' 
                knode = sio.loadmat(path_X_kergm)['full_assignment_mat']
                print('X_KerGM shape', knode.shape)
                
                P = matcheig.matcheig(knode, udim, sizes_dummy)
#                 P = U @ U.T

#                 U = quickmatch.quickmatch(graph_meta.list_graphs, 'coord', 0.26, 0.63)
    
#                 P = U @ U.T

                X_MatchEig = {}
                X_MatchEig['X'] = P
                
                print('P shape', P.shape)
                
                #save with dummy
                sio.savemat(path_to_graph_folder + '/' + trial + '/' + folder + '/X_MatchEig.mat',X_MatchEig)
                
                #remove dummy to compute F1
                X = remove_dummy_rowcol_X(P, list_graphs_dummy)
                print('res shape', res.shape)
                print('X shape', X.shape)
                
                f1, prec, rec = matching.compute_f1score(X,res)
                print('F1: ', f1)

                scores[int(noise)].append(f1)
                precision[int(noise)].append(prec)
                recall[int(noise)].append(rec)

trial:  0
Noise folder:  noise_100,outliers_varied
X_KerGM shape (13563, 13563)
P shape (13563, 13563)
res shape (12085, 12085)
X shape (12085, 12085)
F1:  0.44437857252251584
Noise folder:  noise_400,outliers_varied
X_KerGM shape (13700, 13700)
P shape (13700, 13700)
res shape (12008, 12008)
X shape (12008, 12008)
F1:  0.7675501007312527
Noise folder:  noise_200,outliers_varied
X_KerGM shape (14111, 14111)
P shape (14111, 14111)
res shape (12036, 12036)
X shape (12036, 12036)
F1:  0.6356192703061064
Noise folder:  noise_1000,outliers_varied
X_KerGM shape (13837, 13837)
P shape (13837, 13837)
res shape (12051, 12051)
X shape (12051, 12051)
F1:  0.832895722463416
trial:  1
Noise folder:  noise_100,outliers_varied
X_KerGM shape (13289, 13289)
P shape (13289, 13289)
res shape (12110, 12110)
X shape (12110, 12110)
F1:  0.4609803562111319
Noise folder:  noise_400,outliers_varied
X_KerGM shape (13152, 13152)
P shape (13152, 13152)
res shape (12029, 12029)
X shape (12029, 12029)
F1:  0.761848

In [9]:
import pickle
pickle.dump( precision, open( "precision_match_eig.pickle", "wb" ) )
pickle.dump( scores, open( "scores_match_eig.pickle", "wb" ) )
pickle.dump( recall, open( "recall_match_eig.pickle", "wb" ) )

In [ ]:
p1 = pickle.load( open( "recall_match_eig.pickle", "rb" ) )
p1

In [ ]:
scores
match_eig = scores

In [ ]:
kmeans = {100: [0.633630289532294,
  0.5964303420922161,
  0.6086483546369322,
  0.60486674391657],
 400: [0.7458039406470445,
  0.760277365032194,
  0.6800560485754321,
  0.6219205630970336],
 700: [0.7498272287491362,
  0.7080394922425952,
  0.61010101010101,
  0.6971312515866971],
 1000: [0.7381974248927039,
  0.681999386691199,
  0.6434782608695652,
  0.6336032388663968],
 1300: [0.7844391153743672,
  0.7408305547135495,
  0.6871002132196162,
  0.7225617208506478]}

In [ ]:
quick_match = {100: [0.14484272128749087,
  0.12236842105263158,
  0.1318051575931232,
  0.1385714285714286],
 400: [0.13870776526378187,
  0.13641755634638197,
  0.13211009174311927,
  0.15546218487394958],
 700: [0.16313213703099513,
  0.15613910574875797,
  0.15036496350364964,
  0.14882506527415146],
 1000: [0.14947498455836938,
  0.15445859872611464,
  0.1475529583637692,
  0.15547703180212016],
 1300: [0.1452513966480447,
  0.15882352941176472,
  0.14603616133518776,
  0.13750767341927564]}

In [ ]:
def score_mean_std(scores):
    
    avg_scores = []
    std_scores = []

    for keys,values in scores.items():
        avg_scores.append(np.mean(values))
        std_scores.append(np.std(values))
        
    return np.array(avg_scores), np.array(std_scores)

In [ ]:
kmeans_mean, kmeans_std  = score_mean_std(kmeans)
quick_means, quick_std  = score_mean_std(quick_match)
match_mean, match_std = score_mean_std(match_eig)

In [ ]:
plt.figure(figsize=(14, 8))

plt.plot(list(scores.keys()), kmeans_mean ,label = 'kmeans')
plt.fill_between(list(scores.keys()), kmeans_mean - kmeans_std, kmeans_mean + kmeans_std, alpha=0.2)


plt.plot(list(scores.keys()), quick_means ,label = 'quick_match')
plt.fill_between(list(scores.keys()), quick_means - quick_std, quick_means + quick_std, alpha=0.2)



plt.plot(list(scores.keys()), match_mean ,label = 'match_eig')
plt.fill_between(list(scores.keys()), match_mean - match_std, match_mean + match_std, alpha=0.2)



plt.xlabel('kappa',fontweight="bold")
plt.ylabel('F1 score',fontweight="bold")
plt.legend(loc = 'lower left')
plt.title('kmeans, quickmatch and matcheig on simultion for different kappa values',fontweight="bold")
plt.gca().yaxis.grid(True)
plt.gca().invert_xaxis()
plt.show()

In [ ]:
gt_path = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_test_single_noise/0.0/noise_1000,outliers_varied/ground_truth.gpickle'
dummy_path = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_with_dummy/0.0/noise_1000,outliers_varied/0/graphs/'
non_dummy_path = '/home/rohit/PhD_Work/GM_my_version/Graph_matching/data/simu_graph/simu_test_single_noise/0.0/noise_1000,outliers_varied/'

In [ ]:
cao_kappa_1000 = sio.loadmat('X_cao_cst_o.mat')['X']
mals_kappa_1000 = sio.loadmat('X_mALS.mat')['X']
all_dummy_graphs = [nx.read_gpickle(dummy_path+'/'+g) for g in np.sort(os.listdir(dummy_path))]

In [ ]:
graph_m = GraphDataset(non_dummy_path + '/graphs', non_dummy_path + '/permutation_to_ref_graph.gpickle')
gt = nx.read_gpickle(gt_path)
res = get_permutation_matrix_from_dictionary(gt, graph_m.sizes)

In [ ]:
dummy_mask = [list(nx.get_node_attributes(graph,'is_dummy').values()) for graph in all_dummy_graphs]
dummy_mask = sum(dummy_mask,[])
dummy_indexes = [i for i in range(len(dummy_mask)) if dummy_mask[i]==True]   

In [ ]:
cao = np.delete(cao_kappa_1000,dummy_indexes,0) # delete the dummy rows
cao = np.delete(cao,dummy_indexes,1) # delete the dummy columns

In [ ]:
mAls = np.delete(mals_kappa_1000,dummy_indexes,0) # delete the dummy rows
mAls = np.delete(mAls,dummy_indexes,1) # delete the dummy columns

In [ ]:
matching.compute_f1score(cao,res)

In [ ]:
matching.compute_f1score(mAls,res)